In [ ]:
import subprocess
import sys

def instalar_librerias(librerias):
    for libreria in librerias:
        try:
            # Intentamos importar la librería
            __import__(libreria)
            print(f'La librería {libreria} ya está instalada.')
        except ImportError:
            # Si no se puede importar, la instalamos
            print(f'La librería {libreria} no está instalada. Instalando...')
            subprocess.check_call([sys.executable, "-m", "pip", "install", libreria])

In [ ]:
instalar_librerias(['beautifulsoup4', 'undetected-chromedriver', 'pandas', 'selenium', 'setuptools'])


La librería beautifulsoup4 no está instalada. Instalando...
La librería undetected-chromedriver no está instalada. Instalando...
La librería pandas ya está instalada.
La librería selenium ya está instalada.
La librería setuptools ya está instalada.


In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import random
import re
import time
import pandas as pd
import numpy as np
from selenium import webdriver
from datetime import datetime

from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from datetime import date
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import undetected_chromedriver as uc


# Oeschle Scrapping

In [ ]:
#Código para ingresar a la página de Oeschle

def acceso(browser, url):
    browser.get(url)
    try:
        # Espera explícita hasta que los productos estén presentes
        wait = WebDriverWait(browser, 15)
        wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "text.fz-15.fz-lg-17.prod-name")))
    except:
        print(f"Tiempo de espera agotado para {url}")
    html = browser.page_source
    soup = bs(html, "html.parser")
    return soup


In [ ]:
# Función para extraer texto de elementos
def text_extract(textos):
    return [texto.get_text(strip=True) for texto in textos]

# Función para extraer precios
def prices_extract(precios):
    return [precio.get_text(strip=True)[3:] for precio in precios]

In [ ]:
def unir_listas(lista_de_listas):
    """
    Convierte una lista de listas en una sola lista usando un bucle.

    :param lista_de_listas: Lista de listas a unir.
    :return: Lista unificada.
    """
    resultado = []
    for sublista in lista_de_listas:
        resultado.extend(sublista)
    return resultado

In [ ]:
# Configurar opciones de Chrome
chrome_options = Options()
chrome_options.add_argument("--disable-notifications")  # Desactivar notificaciones

# Inicializar el navegador una sola vez
#browser = uc.Chrome()
browser = webdriver.Chrome(options=chrome_options)
browser.implicitly_wait(10)  # Espera implícita de 10 segundos

# Lista de Urls
lista_url=["https://www.oechsle.pe/zapatillas",
           "https://www.oechsle.pe/moda/mujer",
           "https://www.oechsle.pe/moda/hombre",
           "https://www.oechsle.pe/moda/pijamas-y-ropa-interior/pijamas-para-hombre",
           "https://www.oechsle.pe/moda/pijamas-y-ropa-interior/batas-hombre",
           "https://www.oechsle.pe/moda/pijamas-y-ropa-interior/boxers-hombre",
           "https://www.oechsle.pe/moda/pijamas-y-ropa-interior/pantuflas-hombre",
           "https://www.oechsle.pe/moda/pijamas-y-ropa-interior/boxers-hombre",
           "https://www.oechsle.pe/moda/pijamas-y-ropa-interior/medias-para-hombre",
           "https://www.oechsle.pe/moda/moda-peru/hombre-peru",
           "https://www.oechsle.pe/moda/lenceria-y-pijamas",
           "https://www.oechsle.pe/moda/ropas-de-bano-para-mujer",
           "https://www.oechsle.pe/moda/moda-peru/mujer-peru",
           "https://www.oechsle.pe/zapatos",
           "https://www.oechsle.pe/moda/bebe-nina-0-a-36-meses",
           "https://www.oechsle.pe/moda/bebe-nino-0-a-36-meses",
           "https://www.oechsle.pe/moda/ninas-4-a-12-anos",
           "https://www.oechsle.pe/moda/ninos-4-a-12-anos",
           "https://www.oechsle.pe/moda/ninas-12-a-16-anos",
           "https://www.oechsle.pe/moda/ninos-12-a-16-anos"
          ]

# Listas para almacenar los datos
items_categoria=[]
marcas_categoria=[]
precios_categoria=[]
tipo_categoria=[]

# Definir la expresión regular para la clase
pattern = re.compile(r'texto\s+brand\s+\w+')

for url in lista_url:

    # Contador de páginas
    pagina = 1
    base = url

    while True :

        print(f"Procesando página {pagina}: {url}")

        # Obtener el contenido de la página
        soup = acceso(browser, url)

        #Encuentra los items, marcas y precios en el html de la página i
        items=soup.find_all("span", {"class":"text fz-15 fz-lg-17 prod-name"})
        # Buscar todos los <p> que coincidan con la expresión regular en la clase
        marcas = soup.find_all("p", class_=pattern)
        #marcas = soup.find_all("p", {"class":"texto brand adidas"})
        precios=soup.find_all("span", {"class":"text fz-lg-15 fw-bold BestPrice"})

        #Extra los items, marcas y precios de la página i
        items_list=text_extract(items)
        marcas_list=text_extract(marcas)
        precios_list=prices_extract(precios)

        # Verificar si se encontraron productos en la página
        if not items_list:
            print("No se encontraron más productos. Finalizando el scraping.")
            break

        #Coloca todos los elementos en una listas globales
        items_categoria.append(items_list)
        marcas_categoria.append(marcas_list)
        precios_categoria.append(precios_list)
        tipo_categoria.extend([base.rstrip('/').split('/')[-1]] * len(items))

        try:
            # Espera explícita hasta que el botón "Siguiente" sea clicable
            wait = WebDriverWait(browser, 10)
            next_button = wait.until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "a.page-link.next"))
            )

            # Obtener la URL de la siguiente página
            url = next_button.get_attribute("href")
            # Hacer clic en el botón "Siguiente"
            #next_button.click()
            pagina += 1

            # Esperar un tiempo aleatorio para evitar ser detectado como bot
            time.sleep(random.uniform(2, 5))

        except TimeoutException:
            print("No se encontró el botón 'Siguiente'. Finalizando el scraping.")
            break
        except NoSuchElementException:
            print("No se encontró el botón 'Siguiente'. Finalizando el scraping.")
            break
        except Exception as e:
            print(f"Ocurrió un error inesperado: {e}")
            break

# Cerrar el navegador después del scraping
browser.close()
browser.quit()

items_categoria=unir_listas(items_categoria)
marcas_categoria=unir_listas(marcas_categoria)
precios_categoria=unir_listas(precios_categoria)
print(len(precios_categoria))
print(len(tipo_categoria))
#tipo_categoria=unir_listas(tipo_categoria)

df=pd.DataFrame({
           "Item":items_categoria,
           "Marca":marcas_categoria,
           "Precio":precios_categoria,
           "Categoria":tipo_categoria
})

df["Web"]="Oeschle"
df["Fecha"]=datetime.now().strftime("%m/%d/%Y")
df

df.to_csv('oeschle.csv', index=False)

Procesando página 1: https://www.oechsle.pe/zapatillas
Procesando página 2: https://www.oechsle.pe/zapatillas?&optionOrderBy=OrderByScoreDESC&O=OrderByScoreDESC&page=2
Procesando página 3: https://www.oechsle.pe/zapatillas?&optionOrderBy=OrderByScoreDESC&optionOrderBy=OrderByScoreDESC&O=OrderByScoreDESC&optionOrderBy=OrderByScoreDESC&page=3
Procesando página 4: https://www.oechsle.pe/zapatillas?&optionOrderBy=OrderByScoreDESC&optionOrderBy=OrderByScoreDESC&O=OrderByScoreDESC&optionOrderBy=OrderByScoreDESC&page=4
Procesando página 5: https://www.oechsle.pe/zapatillas?&optionOrderBy=OrderByScoreDESC&optionOrderBy=OrderByScoreDESC&O=OrderByScoreDESC&optionOrderBy=OrderByScoreDESC&page=5
Procesando página 6: https://www.oechsle.pe/zapatillas?&optionOrderBy=OrderByScoreDESC&optionOrderBy=OrderByScoreDESC&O=OrderByScoreDESC&optionOrderBy=OrderByScoreDESC&page=6
Procesando página 7: https://www.oechsle.pe/zapatillas?&optionOrderBy=OrderByScoreDESC&optionOrderBy=OrderByScoreDESC&O=OrderByScore

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=131.0.6778.86)
Stacktrace:
	GetHandleVerifier [0x00007FF6C3396CF5+28821]
	(No symbol) [0x00007FF6C3303880]
	(No symbol) [0x00007FF6C31A578A]
	(No symbol) [0x00007FF6C317F4F5]
	(No symbol) [0x00007FF6C3226247]
	(No symbol) [0x00007FF6C323ECE2]
	(No symbol) [0x00007FF6C321F0A3]
	(No symbol) [0x00007FF6C31EA778]
	(No symbol) [0x00007FF6C31EB8E1]
	GetHandleVerifier [0x00007FF6C36CFCED+3408013]
	GetHandleVerifier [0x00007FF6C36E745F+3504127]
	GetHandleVerifier [0x00007FF6C36DB63D+3455453]
	GetHandleVerifier [0x00007FF6C345BDFB+835995]
	(No symbol) [0x00007FF6C330EB9F]
	(No symbol) [0x00007FF6C330A854]
	(No symbol) [0x00007FF6C330A9ED]
	(No symbol) [0x00007FF6C32FA1D9]
	BaseThreadInitThunk [0x00007FFE121D4CB0+16]
	RtlUserThreadStart [0x00007FFE13A3EDCB+43]
